In [9]:
# 집합 A = {1,2,3,4,5} 에 대한 관계 행렬을 입력받아
# 동치 관계 판별, 동치류 출력, 각 폐포(반사/대칭/추이) 구현,
# 그리고 추가기능으로 오리지널 추이 폐포와 Warshall 추이 폐포를 비교하는 프로그램

### 반사 관계 판별
def is_reflexive(matrix, n):
    for i in range(n):
        if matrix[i][i] != 1:
            return False
    return True

### 대칭 관계 판별
def is_symmetric(matrix, n):
    for i in range(n):
        for j in range(n):
            if matrix[i][j] != matrix[j][i]:
                return False
    return True

# 추이 관계 여부 판별 함수 (오리지널 방식: 정의 그대로 검사)
def is_transitive(matrix, n):
    # 모든 i, k, j에 대해
    # (i,k) ∈ R, (k,j) ∈ R 이면 (i,j)도 반드시 ∈ R 이어야 함
    for i in range(n):
        for k in range(n):
            if matrix[i][k] == 1:
                for j in range(n):
                    if matrix[k][j] == 1 and matrix[i][j] == 0:
                        # 추이 위반 발견
                        return False
    return True

# 추이 폐포 (오리지널 방식)
# 위반되는 (i,k,j)를 찾으면 (i,j)를 1로 추가하는 작업을
# 더 이상 새로 추가할 쌍이 없을 때까지 반복
def transitive_closure_original(matrix, n):
    closure = [row[:] for row in matrix]
    changed = True

    while changed:
        changed = False
        for i in range(n):
            for k in range(n):
                if closure[i][k] == 1:
                    for j in range(n):
                        if closure[k][j] == 1 and closure[i][j] == 0:
                            closure[i][j] = 1
                            changed = True
    return closure

# 추이 폐포 (Warshall 알고리즘 – 추가기능용)
def transitive_closure_warshall(matrix, n):
    closure = [row[:] for row in matrix]
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if closure[i][k] == 1 and closure[k][j] == 1:
                    closure[i][j] = 1
    return closure

# 반사 폐포
def reflexive_closure(matrix, n):
    closure = [row[:] for row in matrix]
    for i in range(n):
        closure[i][i] = 1
    return closure

# 대칭 폐포
def symmetric_closure(matrix, n):
    closure = [row[:] for row in matrix]
    for i in range(n):
        for j in range(n):
            if closure[i][j] == 1:
                closure[j][i] = 1
    return closure

# 행렬 출력 함수
def print_matrix(matrix):
    for row in matrix:
        print(" ".join(map(str, row)))
    print()

# 동치 관계 여부를 판별하고 결과를 출력하는 함수
# → (동치인지 여부, 반사?, 대칭?, 추이?) 를 함께 반환
def check_equivalence(matrix, n):
    r = is_reflexive(matrix, n)
    s = is_symmetric(matrix, n)
    t = is_transitive(matrix, n)

    print("반사 관계 여부:", "예" if r else "아니오")
    print("대칭 관계 여부:", "예" if s else "아니오")
    print("추이 관계 여부:", "예" if t else "아니오")

    if r and s and t:
        print("→ 이 관계는 동치 관계입니다.")
        is_eq = True
    else:
        print("→ 이 관계는 동치 관계가 아닙니다.")
        is_eq = False

    print()
    return is_eq, r, s, t

# 특정 원소 a에 대한 동치류를 구하는 함수
def equivalence_class(matrix, A, a):
    n = len(A)
    idx = A.index(a)
    eq_class = []
    for j in range(n):
        if matrix[idx][j] == 1:
            eq_class.append(A[j])
    return eq_class

# 전체 원소에 대한 동치류 출력
def print_equivalence_classes(matrix, A):
    print("동치류:")
    for a in A:
        eq = equivalence_class(matrix, A, a)
        print(f"[{a}] = {{{', '.join(map(str, eq))}}}")
    print()

# --- 추가기능: 오리지널 추이 폐포 vs Warshall 추이 폐포 비교 ---
def compare_original_and_warshall(matrix, n):
    orig = transitive_closure_original(matrix, n)
    war  = transitive_closure_warshall(matrix, n)

    print("=== 추가기능: 오리지널 방식과 Warshall 알고리즘 추이 폐포 비교 ===")
    print("오리지널 추이 폐포:")
    print_matrix(orig)
    print("Warshall 추이 폐포:")
    print_matrix(war)

    if orig == war:
        print("→ 두 방식의 추이 폐포 결과가 완전히 동일합니다. ✅")
    else:
        print("→ 두 방식의 추이 폐포 결과가 서로 다릅니다. ❌")
        print("서로 다른 위치(오리지널 != Warshall):")
        for i in range(n):
            for j in range(n):
                if orig[i][j] != war[i][j]:
                    print(f"  위치 ({i}, {j}) : original={orig[i][j]}, warshall={war[i][j]}")
    print()

def main():
    A = [1, 2, 3, 4, 5]
    n = len(A)
    matrix = []

    print("5x5 관계 행렬을 행 단위로 입력하시오 (원소: 0 또는 1): ")
    for _ in range(n):
        row = list(map(int, input().split()))
        matrix.append(row)

    print("\n입력된 관계 행렬:")
    print_matrix(matrix)

    print("원래 관계에 대한 성질 판별:")
    is_eq, r, s, t = check_equivalence(matrix, n)

    if is_eq:
        print_equivalence_classes(matrix, A)

    # ★ 추가기능: 동일한 입력에 대해 오리지널/Warshall 추이 폐포 비교
    compare_original_and_warshall(matrix, n)

    # ===== 여기서부터 '깨진 성질'에 대해서만 폐포 수행 =====

    # 1) 반사 관계가 아닌 경우에만 반사 폐포 적용
    if not r:
        print("※ 입력된 관계가 반사 관계가 아니므로, 반사 폐포로 변환합니다.")
        print("반사 폐포 변환 전 행렬:")
        print_matrix(matrix)

        reflex_cl = reflexive_closure(matrix, n)
        print("반사 폐포 변환 후 행렬:")
        print_matrix(reflex_cl)

        print("반사 폐포에 대한 성질 판별:")
        eq2, r2, s2, t2 = check_equivalence(reflex_cl, n)
        if eq2:
            print_equivalence_classes(reflex_cl, A)

    # 2) 대칭 관계가 아닌 경우에만 대칭 폐포 적용
    if not s:
        print("※ 입력된 관계가 대칭 관계가 아니므로, 대칭 폐포로 변환합니다.")
        print("대칭 폐포 변환 전 행렬:")
        print_matrix(matrix)

        sym_cl = symmetric_closure(matrix, n)
        print("대칭 폐포 변환 후 행렬:")
        print_matrix(sym_cl)

        print("대칭 폐포에 대한 성질 판별:")
        eq3, r3, s3, t3 = check_equivalence(sym_cl, n)
        if eq3:
            print_equivalence_classes(sym_cl, A)

    # 3) 추이 관계가 아닌 경우에만 추이 폐포 적용 (오리지널 방식 사용)
    if not t:
        print("※ 입력된 관계가 추이 관계가 아니므로, 추이 폐포로 변환합니다.")
        print("추이 폐포 변환 전 행렬:")
        print_matrix(matrix)

        trans_cl = transitive_closure_original(matrix, n)
        print("추이 폐포 변환 후 행렬(오리지널 방식):")
        print_matrix(trans_cl)

        print("추이 폐포에 대한 성질 판별:")
        eq4, r4, s4, t4 = check_equivalence(trans_cl, n)
        if eq4:
            print_equivalence_classes(trans_cl, A)

In [6]:
if __name__ == "__main__":
    main()

5x5 관계 행렬을 행 단위로 입력하시오 (원소: 0 또는 1): 
1 1 0 0 0
1 1 0 0 0
0 0 1 1 1
0 0 1 1 1
0 0 1 1 1

입력된 관계 행렬:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 1
0 0 1 1 1
0 0 1 1 1

원래 관계에 대한 성질 판별:
반사 관계 여부: 예
대칭 관계 여부: 예
추이 관계 여부: 예
→ 이 관계는 동치 관계입니다.

동치류:
[1] = {1, 2}
[2] = {1, 2}
[3] = {3, 4, 5}
[4] = {3, 4, 5}
[5] = {3, 4, 5}

=== 추가기능: 오리지널 방식과 Warshall 알고리즘 추이 폐포 비교 ===
오리지널 추이 폐포:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 1
0 0 1 1 1
0 0 1 1 1

Warshall 추이 폐포:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 1
0 0 1 1 1
0 0 1 1 1

→ 두 방식의 추이 폐포 결과가 완전히 동일합니다. ✅



In [7]:
if __name__ == "__main__":
    main()

5x5 관계 행렬을 행 단위로 입력하시오 (원소: 0 또는 1): 
1 1 0 0 0
1 1 1 0 0
0 1 1 0 0
0 0 0 1 0
0 0 0 0 1

입력된 관계 행렬:
1 1 0 0 0
1 1 1 0 0
0 1 1 0 0
0 0 0 1 0
0 0 0 0 1

원래 관계에 대한 성질 판별:
반사 관계 여부: 예
대칭 관계 여부: 예
추이 관계 여부: 아니오
→ 이 관계는 동치 관계가 아닙니다.

=== 추가기능: 오리지널 방식과 Warshall 알고리즘 추이 폐포 비교 ===
오리지널 추이 폐포:
1 1 1 0 0
1 1 1 0 0
1 1 1 0 0
0 0 0 1 0
0 0 0 0 1

Warshall 추이 폐포:
1 1 1 0 0
1 1 1 0 0
1 1 1 0 0
0 0 0 1 0
0 0 0 0 1

→ 두 방식의 추이 폐포 결과가 완전히 동일합니다. ✅

※ 입력된 관계가 추이 관계가 아니므로, 추이 폐포로 변환합니다.
추이 폐포 변환 전 행렬:
1 1 0 0 0
1 1 1 0 0
0 1 1 0 0
0 0 0 1 0
0 0 0 0 1

추이 폐포 변환 후 행렬(오리지널 방식):
1 1 1 0 0
1 1 1 0 0
1 1 1 0 0
0 0 0 1 0
0 0 0 0 1

추이 폐포에 대한 성질 판별:
반사 관계 여부: 예
대칭 관계 여부: 예
추이 관계 여부: 예
→ 이 관계는 동치 관계입니다.

동치류:
[1] = {1, 2, 3}
[2] = {1, 2, 3}
[3] = {1, 2, 3}
[4] = {4}
[5] = {5}



In [8]:
if __name__ == "__main__":
    main()

5x5 관계 행렬을 행 단위로 입력하시오 (원소: 0 또는 1): 
1 1 0 0 0
1 1 0 0 0
0 0 1 1 0 
0 0 1 1 0
0 0 0 0 0

입력된 관계 행렬:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 0
0 0 1 1 0
0 0 0 0 0

원래 관계에 대한 성질 판별:
반사 관계 여부: 아니오
대칭 관계 여부: 예
추이 관계 여부: 예
→ 이 관계는 동치 관계가 아닙니다.

=== 추가기능: 오리지널 방식과 Warshall 알고리즘 추이 폐포 비교 ===
오리지널 추이 폐포:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 0
0 0 1 1 0
0 0 0 0 0

Warshall 추이 폐포:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 0
0 0 1 1 0
0 0 0 0 0

→ 두 방식의 추이 폐포 결과가 완전히 동일합니다. ✅

※ 입력된 관계가 반사 관계가 아니므로, 반사 폐포로 변환합니다.
반사 폐포 변환 전 행렬:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 0
0 0 1 1 0
0 0 0 0 0

반사 폐포 변환 후 행렬:
1 1 0 0 0
1 1 0 0 0
0 0 1 1 0
0 0 1 1 0
0 0 0 0 1

반사 폐포에 대한 성질 판별:
반사 관계 여부: 예
대칭 관계 여부: 예
추이 관계 여부: 예
→ 이 관계는 동치 관계입니다.

동치류:
[1] = {1, 2}
[2] = {1, 2}
[3] = {3, 4}
[4] = {3, 4}
[5] = {5}

